# Trabalho Prático 3

## Alunos:
### Henrique Tadashi Tarzia - 10692210
### Luís Felipe Ribeiro Chaves - 10801221

## Atualizações da parte 3:

 - Declaração de vetor normal no código de vértice

 - Inclusão de duas fontes de luz no código de fragmento com possibilidade de modificação da cor emitida

 - Coleta dos vetores normais durante a leitura do arquivo na função "load_model_from_file"

 - Criação de uma lista global para armazenamento dos vetores normais dos modelos carregados (normals_list)

 - Envio dos dados relacionados aos vetores normais à GPU (ver em tópico "Dados de Iluminação: vetores normais")

 - Criação de uma função para inserir objetos no cenário, especificando propriedades de iluminação (draw_object_ilum)

 - Adição da variável "ia" no shader de fragmento para realizar o controle da intensidade da luz ambiente


### Primeiro, vamos importar as bibliotecas necessárias.
Verifique no código anterior um script para instalar as dependências necessárias (OpenGL e GLFW) antes de prosseguir.

In [142]:
import glfw
from OpenGL.GL import *
import OpenGL.GL.shaders
import numpy as np
import glm
import math
from PIL import Image

### Inicializando janela

In [143]:
glfw.init()
glfw.window_hint(glfw.VISIBLE, glfw.FALSE);
altura = 1080
largura = 1920
window = glfw.create_window(largura, altura, "SCC0250 - Trabalho 3", None, None)
glfw.make_context_current(window)

### GLSL (OpenGL Shading Language)

Aqui veremos nosso primeiro código GLSL.

É uma linguagem de shading de alto nível baseada na linguagem de programação C.

Nós estamos escrevendo código GLSL como se "strings" de uma variável (mas podemos ler de arquivos texto). Esse código, depois, terá que ser compilado e linkado ao nosso programa. 

Iremos aprender GLSL conforme a necessidade do curso. Usarmos uma versão do GLSL mais antiga, compatível com muitos dispositivos.

### GLSL para Vertex Shader

No Pipeline programável, podemos interagir com Vertex Shaders.

No código abaixo, estamos fazendo o seguinte:

* Definindo uma variável chamada position do tipo vec3.
* Definindo matrizes Model, View e Projection que acumulam transformações geométricas 3D e permitem navegação no cenário.
* void main() é o ponto de entrada do nosso programa (função principal)
* gl_Position é uma variável especial do GLSL. Variáveis que começam com 'gl_' são desse tipo. Nesse caso, determina a posição de um vértice. Observe que todo vértice tem 4 coordenadas, por isso nós combinamos nossa variável vec2 com uma variável vec4. Além disso, nós modificamos nosso vetor com base nas transformações Model, View e Projection.

In [144]:
vertex_code = """
        attribute vec3 position;
        attribute vec2 texture_coord;
        attribute vec3 normals;     
       
        varying vec2 out_texture;
        varying vec3 out_fragPos;
        varying vec3 out_normal;
                
        uniform mat4 model;
        uniform mat4 view;
        uniform mat4 projection;        
        
        void main(){
            gl_Position = projection * view * model * vec4(position,1.0);
            out_texture = vec2(texture_coord);
            out_fragPos = vec3(model * vec4(position, 1.0));
            out_normal = normals;            
        }
        """

### GLSL para Fragment Shader

No Pipeline programável, podemos interagir com Fragment Shaders.

No código abaixo, estamos fazendo o seguinte:

* void main() é o ponto de entrada do nosso programa (função principal)
* gl_FragColor é uma variável especial do GLSL. Variáveis que começam com 'gl_' são desse tipo. Nesse caso, determina a cor de um fragmento. Nesse caso é um ponto, mas poderia ser outro objeto (ponto, linha, triangulos, etc).

### Possibilitando modificar a cor.

Nos exemplos anteriores, a variável gl_FragColor estava definida de forma fixa (com cor R=0, G=0, B=0).

Agora, nós vamos criar uma variável do tipo "uniform", de quatro posições (vec4), para receber o dado de cor do nosso programa rodando em CPU.

In [145]:
fragment_code = """

        // parametros da iluminacao ambiente e difusa
        uniform vec3 lightPos1; // define coordenadas de posicao da luz #1
        uniform vec3 lightPos2; // define coordenadas de posicao da luz #2
        uniform float ka;       // coeficiente de reflexao ambiente
        uniform float ia;       // intensidade da luz ambiente
        uniform float kd;       // coeficiente de reflexao difusa
        uniform float il1;       // intensidade de luz puntual da luz #1
        
        // parametros da iluminacao especular
        uniform vec3 viewPos; // define coordenadas com a posicao da camera/observador
        uniform float ks;     // coeficiente de reflexao especular
        uniform float ns;     // expoente de reflexao especular
        
        // parametro com a cor da(s) fonte(s) de iluminacao
        vec3 lightColor = vec3(1.0, 1.0, 1.0);

        // parametros recebidos do vertex shader
        varying vec2 out_texture;   // recebido do vertex shader
        varying vec3 out_normal;    // recebido do vertex shader
        varying vec3 out_fragPos;   // recebido do vertex shader
        uniform sampler2D samplerTexture;
        
        void main(){
        
            // calculando reflexao ambiente
            vec3 ambient = ka * ia * lightColor;             
        
            ////////////////////////
            // Luz #1
            ////////////////////////
            
            // calculando reflexao difusa
            vec3 norm1 = normalize(out_normal);                  // normaliza vetores perpendiculares
            vec3 lightDir1 = normalize(lightPos1 - out_fragPos); // direcao da luz
            float diff1 = max(dot(norm1, lightDir1), 0.0);       // verifica limite angular (entre 0 e 90)
            vec3 diffuse1 = kd * il1 * diff1 * lightColor;       // iluminacao difusa
            
            // calculando reflexao especular
            vec3 viewDir1 = normalize(viewPos - out_fragPos);    // direcao do observador/camera
            vec3 reflectDir1 = reflect(-lightDir1, norm1);       // direcao da reflexao
            float spec1 = pow(max(dot(viewDir1, reflectDir1), 0.0), ns);
            vec3 specular1 = ks * spec1 * lightColor;        
            
            ////////////////////////
            // Luz #2
            ////////////////////////
            
            // calculando reflexao difusa
            vec3 norm2 = normalize(out_normal);                     // normaliza vetores perpendiculares
            vec3 lightDir2 = normalize(lightPos2 - out_fragPos);    // direcao da luz
            float diff2 = max(dot(norm2, lightDir2), 0.0);          // verifica limite angular (entre 0 e 90)
            vec3 diffuse2 = kd * diff2 * lightColor;                // iluminacao difusa
            
            // calculando reflexao especular
            vec3 viewDir2 = normalize(viewPos - out_fragPos);       // direcao do observador/camera
            vec3 reflectDir2 = reflect(-lightDir2, norm2);          // direcao da reflexao
            float spec2 = pow(max(dot(viewDir2, reflectDir2), 0.0), ns);
            vec3 specular2 = ks * spec2 * lightColor;    
            
            ////////////////////////
            // Combinando as duas fontes
            ////////////////////////
            
            // aplicando o modelo de iluminacao
            vec4 texture = texture2D(samplerTexture, out_texture);
            //vec4 result = vec4((ambient + diffuse1 + diffuse2 + specular1 + specular2),1.0) * texture; // aplica iluminacao
            vec4 result = vec4((ambient + diffuse1),1.0) * texture; // aplica iluminacao
            gl_FragColor = result;

        }
        """

### Requisitando slot para a GPU para nossos programas Vertex e Fragment Shaders

In [146]:
# Request a program and shader slots from GPU
program  = glCreateProgram()
vertex   = glCreateShader(GL_VERTEX_SHADER)
fragment = glCreateShader(GL_FRAGMENT_SHADER)


### Associando nosso código-fonte aos slots solicitados

In [147]:
# Set shaders source
glShaderSource(vertex, vertex_code)
glShaderSource(fragment, fragment_code)

### Compilando o Vertex Shader

Se há algum erro em nosso programa Vertex Shader, nosso app para por aqui.

In [148]:
# Compile shaders
glCompileShader(vertex)
if not glGetShaderiv(vertex, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(vertex).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Vertex Shader")


### Compilando o Fragment Shader

Se há algum erro em nosso programa Fragment Shader, nosso app para por aqui.

In [149]:
glCompileShader(fragment)
if not glGetShaderiv(fragment, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(fragment).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Fragment Shader")

### Associando os programas compilado ao programa principal

In [150]:
# Attach shader objects to the program
glAttachShader(program, vertex)
glAttachShader(program, fragment)


### Linkagem do programa

In [151]:
# Build program
glLinkProgram(program)
if not glGetProgramiv(program, GL_LINK_STATUS):
    print(glGetProgramInfoLog(program))
    raise RuntimeError('Linking error')
    
# Make program the default program
glUseProgram(program)

### Preparando dados para enviar a GPU

Nesse momento, nós compilamos nossos Vertex e Program Shaders para que a GPU possa processá-los.

Por outro lado, as informações de vértices geralmente estão na CPU e devem ser transmitidas para a GPU.


### Carregando Modelos (vértices e texturas) a partir de Arquivos

A função abaixo carrega modelos a partir de arquivos no formato WaveFront.


Para saber mais sobre o modelo, acesse: https://en.wikipedia.org/wiki/Wavefront_.obj_file


Nos slides e vídeo-aula da Aula 11 - Parte 1, nós descrevemos o funcionamento desse formato.

In [152]:
def load_model_from_file(filename):
    """Loads a Wavefront OBJ file. """
    objects = {}
    vertices = []
    normals = []
    texture_coords = []
    faces = []

    material = None

    # abre o arquivo obj para leitura
    for line in open(filename, "r"): ## para cada linha do arquivo .obj
        if line.startswith('#'): continue ## ignora comentarios
        values = line.split() # quebra a linha por espaço
        if not values: continue


        ### recuperando vertices
        if values[0] == 'v':
            vertices.append(values[1:4])

        ### recuperando vertices
        if values[0] == 'vn':
            normals.append(values[1:4])

        ### recuperando coordenadas de textura
        elif values[0] == 'vt':
            texture_coords.append(values[1:3])

        ### recuperando faces 
        elif values[0] in ('usemtl', 'usemat'):
            material = values[1]
        elif values[0] == 'f':
            face = []
            face_texture = []
            face_normals = []
            for v in values[1:]:
                w = v.split('/')
                face.append(int(w[0]))
                face_normals.append(int(w[2]))
                if len(w) >= 2 and len(w[1]) > 0:
                    face_texture.append(int(w[1]))
                else:
                    face_texture.append(0)

            faces.append((face, face_texture, face_normals, material))

    model = {}
    model['vertices'] = vertices
    model['texture'] = texture_coords
    model['faces'] = faces
    model['normals'] = normals

    return model


In [153]:
glEnable(GL_TEXTURE_2D)
qtd_texturas = 10
textures = glGenTextures(qtd_texturas)

def load_texture_from_file(texture_id, img_textura):
    glBindTexture(GL_TEXTURE_2D, texture_id)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_WRAP_S, GL_REPEAT)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_WRAP_T, GL_REPEAT)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_MIN_FILTER, GL_LINEAR)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_MAG_FILTER, GL_LINEAR)
    img = Image.open(img_textura)
    img_width = img.size[0]
    img_height = img.size[1]
    image_data = img.tobytes("raw", "RGB", 0, -1)
    #image_data = np.array(list(img.getdata()), np.uint8)
    glTexImage2D(GL_TEXTURE_2D, 0, GL_RGB, img_width, img_height, 0, GL_RGB, GL_UNSIGNED_BYTE, image_data)

    return
    

### A lista abaixo armazena todos os vertices carregados dos arquivos

In [154]:
id_textura = 0
vertices_list = []    
normals_list = [] 
textures_coord_list = []

### Vamos carregar cada modelo e definir funções para desenhá-los

In [155]:
# Insere modelo contido no arquivo "file_name" na lista de vértices
def insert_model(file_name, texture_name):
     global id_textura, vertices_list, textures_coord_list, normals_list

     # atribuindo o id do objeto de acordo com o valor atual
     object_id = id_textura
     id_textura += 1 

     modelo = load_model_from_file(file_name)

     vi = len(vertices_list)

     ### inserindo vértices do modelo no vetor de vértices
     print('Processando modelo ' + str(file_name) + ' ...')
     print('Vértice inicial: ' + str(len(vertices_list)))

     for face in modelo['faces']:
          for vertice_id in face[0]:
               vertices_list.append(modelo['vertices'][vertice_id-1])
          for texture_id in face[1]:
               textures_coord_list.append(modelo['texture'][texture_id-1])
          for normal_id in face[2]:
               normals_list.append(modelo['normals'][normal_id-1]) 

     print('Vértice final: ' + str(len(vertices_list)))

     vf = len(vertices_list)

     load_texture_from_file(object_id,texture_name)

     return vi,(vf-vi),object_id


In [156]:
# Insere modelo contido com múltiplas texturas no arquivo "file_name" na lista de vértices
def insert_model_mult_texture(file_name,texture_names):
    global id_textura, vertices_list, textures_coord_list

    n = len(texture_names)      # número de arquivos de textura
    object_id = []              # lista de ids atribuídos às partes do objeto
    vi = []                     # lista de vértices iniciais dos componentes do objeto
    qtd = []                    # lista da quantidade de vértices de cada componente do objeto
    faces_visited = []          # lista de faces visitadas
    component = 0               # número de componentes

     # atribuindo o id do objeto de acordo com o valor atual e o número de arquivos de textura
    for i in range(0,n):
        object_id.append(id_textura)
        id_textura += 1

    modelo = load_model_from_file(file_name) 

    ### inserindo vértices do modelo no vetor de vértices
    print('Processando modelo ' + str(file_name) + ' ...')

    for face in modelo['faces']:
        if face[2] not in faces_visited:
            print('Componente encontrado no vértice ' + str(len(vertices_list)))
            vi.append(len(vertices_list))
            faces_visited.append(face[2])
            component += 1
        if component > 1:
            qtd.append(len(vertices_list) - vi[component - 2])
        for vertice_id in face[0]:
            vertices_list.append( modelo['vertices'][vertice_id-1] )
        for texture_id in face[1]:
            textures_coord_list.append( modelo['texture'][texture_id-1] )
        for normal_id in face[2]:
            normals_list.append(modelo['normals'][normal_id-1])     

    print('Vértice final: ' + str(len(vertices_list)))

    qtd.append(len(vertices_list) - vi[component - 1])      

    for i in range(0,n):
        load_texture_from_file(object_id[i],texture_names[i])

    return vi,qtd,object_id
    

In [157]:
# Função para retornar lista com três elementos
def triple(first,second,third):

    lista = [None] * 3
    lista[0] = first
    lista[1] = second
    lista[2] = third

    return lista

In [158]:

#-----------SKY------------#

angle_sky = 0.0                                                                                       # ângulo de rotação
r_sky = [0.0, 1.0, 0.0]                                                                               # rotação
t_sky = [-25.0, -25.0, -25.0]                                                                         # translação
s_sky = [50.0, 50.0, 50.0]                                                                            # escala
ka_sky = 1.0                        # coeficiente de reflexão ambiente 
kd_sky = 0.0                        # coeficiente de reflexão difusa 
ks_sky = 0.0                        # coeficiente de reflexão especular 
ns_sky = 1.0                        # expoente de reflexao especular
vi_sky, qtd_sky, id_sky = insert_model('modelos/sky/cubo.obj','modelos/sky/sky_modified.png')         # vértices

Processando modelo modelos/sky/cubo.obj ...
Vértice inicial: 0
Vértice final: 36


In [159]:

#---------TERRAIN (GRASS)----------#

angle_grass = 0.0                                                                                       # ângulo de rotação
r_grass = [0.0, 1.0, 0.0]                                                                               # rotação
t_grass = [0.0, -1.0, -12.0]                                                                            # translação
s_grass = [27.0, 27.0, 27.0]                                                                            # escala
ka_grass = 1.0                        # coeficiente de reflexão ambiente 
kd_grass = 1.0                        # coeficiente de reflexão difusa 
ks_grass = 0.2                        # coeficiente de reflexão especular 
ns_grass = 1.0                        # expoente de reflexao especular
vi_grass, qtd_grass, id_grass = insert_model('modelos/terrain/terrain.obj','modelos/terrain/grass.jpg') # vértices

#---------TERRAIN (STREET)----------#

angle_street = 0.0                                                                                             # ângulo de rotação
r_street = [0.0, 1.0, 0.0]                                                                                     # rotação
t_street = [0.0, -1.0, 20.0]                                                                                   # translação
s_street = [28.0, 1.0, 5.0]                                                                                    # escala
ka_street = 1.0                                                                                                # coeficiente de reflexão ambiente 
kd_street = 1.0                                                                                                # coeficiente de reflexão difusa 
ks_street = 0.5                                                                                                # coeficiente de reflexão especular 
ns_street = 16.0                                                                                               # expoente de reflexao especular
vi_street, qtd_street, id_street = insert_model('modelos/terrain/terrain-02.obj','modelos/terrain/street.jpg') # vértices

Processando modelo modelos/terrain/terrain.obj ...
Vértice inicial: 36
Vértice final: 42
Processando modelo modelos/terrain/terrain-02.obj ...
Vértice inicial: 42
Vértice final: 48


In [160]:

#-------------BIRDBATH-------------#

angle_birdbath = 0.0                                                                                                   # ângulo de rotação
r_birdbath = triple(0.0, 1.0, 0.0)                                                                                     # rotação
t_birdbath = triple(8.0, -1.0, 0.0)                                                                                    # translação
s_birdbath = triple(2.0,2.0,2.0)                                                                                       # escala
ka_birdbath = 0.9                        # coeficiente de reflexão ambiente 
kd_birdbath = 0.9                        # coeficiente de reflexão difusa 
ks_birdbath = 0.2                        # coeficiente de reflexão especular 
ns_birdbath = 32.0                       # expoente de reflexao especular
vi_birdbath, qtd_birdbath, id_birdbath = insert_model('modelos/birdbath/birdbath.obj','modelos/birdbath/birdbath.jpg') # vértices

Processando modelo modelos/birdbath/birdbath.obj ...
Vértice inicial: 48
Vértice final: 624


In [161]:

#-------------EAGLE-------------#

angle_eagle = 0.0                                                                                 # ângulo de rotação
r_eagle = triple(0.0,1.0,0.0)                                                                     # rotação
t_eagle = triple(3.0,7.0,0.0)                                                                     # translação
s_eagle = triple(1.0,1.0,1.0)                                                                     # escala
ka_eagle = 0.85                                                                                   # coeficiente de reflexão ambiente 
kd_eagle = 0.85                                                                                   # coeficiente de reflexão difusa 
ks_eagle = 0.85                                                                                   # coeficiente de reflexão especular 
ns_eagle = 64.0                                                                                   # expoente de reflexao especular
vi_eagle, qtd_eagle, id_eagle = insert_model('modelos/eagle/eagle.obj','modelos/eagle/eagle.png') # vértices

Processando modelo modelos/eagle/eagle.obj ...
Vértice inicial: 624
Vértice final: 8157


In [162]:

#-------------TREE-------------#

textures_tree = []
textures_tree.append('modelos/tree/trunk.png')
textures_tree.append('modelos/tree/leaves.png')

angle_tree = 0.0                                                                              # ângulo de rotação
r_tree = [0.0, 1.0, 0.0]                                                                      # rotação
t_tree = [6.0, 3.0, -4.0]                                                                     # translação
s_tree = [200.0, 200.0, 200.0]                                                                # escala
ka_tree = 1.0                                                                                 # coeficiente de reflexão ambiente 
kd_tree = 1.0                                                                                 # coeficiente de reflexão difusa 
ks_tree = 1.0                                                                                 # coeficiente de reflexão especular 
ns_tree = 4.0                                                                                 # expoente de reflexao especular
vi_tree, qtd_tree, id_tree = insert_model_mult_texture('modelos/tree/tree.obj',textures_tree) # vértices

Processando modelo modelos/tree/tree.obj ...
Componente encontrado no vértice 8157
Componente encontrado no vértice 8160
Componente encontrado no vértice 8163
Componente encontrado no vértice 8166
Componente encontrado no vértice 8169
Componente encontrado no vértice 8172
Componente encontrado no vértice 8175
Componente encontrado no vértice 8178
Componente encontrado no vértice 8181
Componente encontrado no vértice 8184
Componente encontrado no vértice 8187
Componente encontrado no vértice 8190
Componente encontrado no vértice 8193
Componente encontrado no vértice 8196
Componente encontrado no vértice 8199
Componente encontrado no vértice 8202
Componente encontrado no vértice 8205
Componente encontrado no vértice 8208
Componente encontrado no vértice 8211
Componente encontrado no vértice 8214
Componente encontrado no vértice 8217
Componente encontrado no vértice 8220
Componente encontrado no vértice 8223
Componente encontrado no vértice 8226
Componente encontrado no vértice 8229
Compo

Componente encontrado no vértice 12168
Componente encontrado no vértice 12171
Componente encontrado no vértice 12174
Componente encontrado no vértice 12177
Componente encontrado no vértice 12180
Componente encontrado no vértice 12183
Componente encontrado no vértice 12186
Componente encontrado no vértice 12189
Componente encontrado no vértice 12192
Componente encontrado no vértice 12195
Componente encontrado no vértice 12198
Componente encontrado no vértice 12201
Componente encontrado no vértice 12204
Componente encontrado no vértice 12207
Componente encontrado no vértice 12210
Componente encontrado no vértice 12213
Componente encontrado no vértice 12216
Componente encontrado no vértice 12219
Componente encontrado no vértice 12222
Componente encontrado no vértice 12225
Componente encontrado no vértice 12228
Componente encontrado no vértice 12231
Componente encontrado no vértice 12234
Componente encontrado no vértice 12237
Componente encontrado no vértice 12240
Componente encontrado no 

Componente encontrado no vértice 13230
Componente encontrado no vértice 13233
Componente encontrado no vértice 13236
Componente encontrado no vértice 13239
Componente encontrado no vértice 13242
Componente encontrado no vértice 13245
Componente encontrado no vértice 13248
Componente encontrado no vértice 13251
Componente encontrado no vértice 13254
Componente encontrado no vértice 13257
Componente encontrado no vértice 13260
Componente encontrado no vértice 13263
Componente encontrado no vértice 13266
Componente encontrado no vértice 13269
Componente encontrado no vértice 13272
Componente encontrado no vértice 13275
Componente encontrado no vértice 13278
Componente encontrado no vértice 13281
Componente encontrado no vértice 13284
Componente encontrado no vértice 13287
Componente encontrado no vértice 13290
Componente encontrado no vértice 13293
Componente encontrado no vértice 13296
Componente encontrado no vértice 13299
Componente encontrado no vértice 13302
Componente encontrado no 

Componente encontrado no vértice 14055
Componente encontrado no vértice 14058
Componente encontrado no vértice 14061
Componente encontrado no vértice 14064
Componente encontrado no vértice 14067
Componente encontrado no vértice 14070
Componente encontrado no vértice 14073
Componente encontrado no vértice 14076
Componente encontrado no vértice 14079
Componente encontrado no vértice 14082
Componente encontrado no vértice 14085
Componente encontrado no vértice 14088
Componente encontrado no vértice 14091
Componente encontrado no vértice 14094
Componente encontrado no vértice 14097
Componente encontrado no vértice 14100
Componente encontrado no vértice 14103
Componente encontrado no vértice 14106
Componente encontrado no vértice 14109
Componente encontrado no vértice 14112
Componente encontrado no vértice 14115
Componente encontrado no vértice 14118
Componente encontrado no vértice 14121
Componente encontrado no vértice 14124
Componente encontrado no vértice 14127
Componente encontrado no 

Componente encontrado no vértice 16407
Componente encontrado no vértice 16410
Componente encontrado no vértice 16413
Componente encontrado no vértice 16416
Componente encontrado no vértice 16419
Componente encontrado no vértice 16422
Componente encontrado no vértice 16425
Componente encontrado no vértice 16428
Componente encontrado no vértice 16431
Componente encontrado no vértice 16434
Componente encontrado no vértice 16437
Componente encontrado no vértice 16440
Componente encontrado no vértice 16443
Componente encontrado no vértice 16446
Componente encontrado no vértice 16449
Componente encontrado no vértice 16452
Componente encontrado no vértice 16455
Componente encontrado no vértice 16458
Componente encontrado no vértice 16461
Componente encontrado no vértice 16464
Componente encontrado no vértice 16467
Componente encontrado no vértice 16470
Componente encontrado no vértice 16473
Componente encontrado no vértice 16476
Componente encontrado no vértice 16479
Componente encontrado no 

Componente encontrado no vértice 17469
Componente encontrado no vértice 17472
Componente encontrado no vértice 17475
Componente encontrado no vértice 17478
Componente encontrado no vértice 17481
Componente encontrado no vértice 17484
Componente encontrado no vértice 17487
Componente encontrado no vértice 17490
Componente encontrado no vértice 17493
Componente encontrado no vértice 17496
Componente encontrado no vértice 17499
Componente encontrado no vértice 17502
Componente encontrado no vértice 17505
Componente encontrado no vértice 17508
Componente encontrado no vértice 17511
Componente encontrado no vértice 17514
Componente encontrado no vértice 17517
Componente encontrado no vértice 17520
Componente encontrado no vértice 17523
Componente encontrado no vértice 17526
Componente encontrado no vértice 17529
Componente encontrado no vértice 17532
Componente encontrado no vértice 17535
Componente encontrado no vértice 17538
Componente encontrado no vértice 17541
Componente encontrado no 

In [163]:

#-------------MUSEUM-------------#

angle_museum = 0.0                                                                                          # ângulo de rotação
r_museum = triple(0.0, 1.0, 0.0)                                                                            # rotação
t_museum = triple(-5.0, 0.7, 0.0)                                                                           # translação
s_museum = triple(0.1,0.1,0.1)                                                                              # escala
ka_museum = 1.0                                                                                             # coeficiente de reflexão ambiente 
kd_museum = 1.0                                                                                             # coeficiente de reflexão difusa 
ks_museum = 0.0                                                                                             # coeficiente de reflexão especular 
ns_museum = 16.0                                                                                            # expoente de reflexao especular
vi_museum, qtd_museum, id_museum = insert_model('modelos/museum/museum.obj','modelos/museum/museum.jpeg')   # vértices

#-------FLOOR (MUSEUM)-------#

angle_floor_m = 0.0                                                                                                 # ângulo de rotação
r_floor_m = triple(0.0, 1.0, 0.0)                                                                                   # rotação
t_floor_m = triple(-5.0, -0.55, 0.0)                                                                                # translação
s_floor_m = triple(6.25,1,3.9)                                                                                      # escala
ka_floor_m = 1.0                                                                                                    # coeficiente de reflexão ambiente 
kd_floor_m = 1.0                                                                                                    # coeficiente de reflexão difusa 
ks_floor_m = 1.0                                                                                                    # coeficiente de reflexão especular 
ns_floor_m = 16.0                                                                                                   # expoente de reflexao especular
vi_floor_m, qtd_floor_m, id_floor_m = insert_model('modelos/floor/floor.obj','modelos/floor/brick_hexagonal.png')   # vértices

Processando modelo modelos/museum/museum.obj ...
Vértice inicial: 20106
Vértice final: 20418
Processando modelo modelos/floor/floor.obj ...
Vértice inicial: 20418
Vértice final: 20424


In [164]:

#-------TABLE-------#

angle_table = 0.0                                                                                   # ângulo de rotação
r_table = triple(0.0, 1.0, 0.0)                                                                     # rotação
t_table = triple(0.8,-0.55,0.5)                                                                     # translação
s_table = triple(1.0,1.0,1.0)                                                                       # escala
ka_table = 0.8                        # coeficiente de reflexão ambiente 
kd_table = 0.8                        # coeficiente de reflexão difusa 
ks_table = 0.2                        # coeficiente de reflexão especular 
ns_table = 16.0                       # expoente de reflexao especular
vi_table, qtd_table, id_table = insert_model('modelos/table/table.obj','modelos/table/table.png')   # vértices

#-------POKEBALL-------#

angle_pokeball = 0.0                                                                                               # ângulo de rotação
r_pokeball = triple(0.0, 1.0, 0.0)                                                                                 # rotação
t_pokeball = triple(-6.0,0.2,-5.2)                                                                                 # translação
s_pokeball = triple(0.03,0.03,0.03)                                                                                # escala
ka_pokeball = 1.0                        # coeficiente de reflexão ambiente 
kd_pokeball = 1.0                        # coeficiente de reflexão difusa 
ks_pokeball = 1.0                        # coeficiente de reflexão especular 
ns_pokeball = 2048.0                     # expoente de reflexao especular
vi_pokeball, qtd_pokeball, id_pokeball = insert_model('modelos/table/pokeball.obj','modelos/table/pokeball.png')   # vértices

Processando modelo modelos/table/table.obj ...
Vértice inicial: 20424
Vértice final: 22530
Processando modelo modelos/table/pokeball.obj ...
Vértice inicial: 22530
Vértice final: 25608


In [165]:

#-------------DUCK-------------#

angle_duck = 0.0                     # ângulo de rotação
r_duck = triple(0.0, 1.0, 0.0)       # rotação
t_duck = triple(-5.0,0.1, 0.0)       # translação
s_duck = triple(0.28,0.28,0.28)      # escala
ka_duck = 0.7                        # coeficiente de reflexão ambiente 
kd_duck = 0.7                        # coeficiente de reflexão difusa 
ks_duck = 0.5                        # coeficiente de reflexão especular 
ns_duck = 128.0                      # expoente de reflexao especular
vi_duck, qtd_duck, id_duck = insert_model('modelos/duck/duck.obj','modelos/duck/duck.png')   # vértices

Processando modelo modelos/duck/duck.obj ...
Vértice inicial: 25608
Vértice final: 88776


In [166]:

#-------------SEEDS-------------#

angle_seeds = 0.0                    # ângulo de rotação
r_seeds = triple(0.0,1.0, 0.0)       # rotação
t_seeds = triple(4.0,-1.0,-2.0)      # translação
s_seeds = triple(12.0,12.0,12.0)     # escala
ka_seeds = 0.6                       # coeficiente de reflexão ambiente 
kd_seeds = 0.6                       # coeficiente de reflexão difusa 
ks_seeds = 0.4                       # coeficiente de reflexão especular 
ns_seeds = 64.0                      # expoente de reflexao especular
vi_seeds, qtd_seeds, id_seeds = insert_model('modelos/seeds/seeds.obj','modelos/seeds/seeds.png') # vértices

Processando modelo modelos/seeds/seeds.obj ...
Vértice inicial: 88776
Vértice final: 90540


In [167]:

#-------------APPLES-------------#

angle_apple = triple(10.0,0.0,15.0)                                                               # ângulo de rotação
r_apple = triple([1.0,0.0,1.0],[0.0,0.0,1.0],[0.0,0.0,1.0])                                       # rotação                                                                    
t_apple = triple([3.6,-1.4,0.2],[4.2,-1.0,-0.2],[3.6,-2.0,-0.5])                                  # translação
s_apple = triple([4.0,4.0,4.0],[4.0,4.0,4.0],[4.0,4.0,4.0])                                       # escala
ka_apple = 0.8                                                                                    # coeficiente de reflexão ambiente 
kd_apple = 0.8                                                                                    # coeficiente de reflexão difusa 
ks_apple = 0.7                                                                                    # coeficiente de reflexão especular 
ns_apple = 256.0                                                                                  # expoente de reflexao especular
vi_apple, qtd_apple, id_apple = insert_model('modelos/apple/apple.obj','modelos/apple/apple.png') # vértices

Processando modelo modelos/apple/apple.obj ...
Vértice inicial: 90540
Vértice final: 91770


In [168]:

#-------------CAMPFIRE-------------#

angle_campfire = 0.0                                                                                                   # ângulo de rotação
r_campfire = triple(0.0,1.0,0.0)                                                                                       # rotação
t_campfire = triple(8.0,-1.0,7.0)                                                                                      # translação
s_campfire = triple(0.02,0.02,0.02)                                                                                    # escala
ka_campfire = 0.4                                                                                                      # coeficiente de reflexão ambiente 
kd_campfire = 0.9                                                                                                      # coeficiente de reflexão difusa 
ks_campfire = 0.3                                                                                                      # coeficiente de reflexão especular 
ns_campfire = 64.0                                                                                                     # expoente de reflexao especular
vi_campfire, qtd_campfire, id_campfire = insert_model('modelos/campfire/campfire.obj','modelos/campfire/campfire.png') # vértices

Processando modelo modelos/campfire/campfire.obj ...
Vértice inicial: 91770
Vértice final: 94752


In [169]:

#-------ALLEGORIE SCULPTURE-------#

angle_allegorie = 0.0                 # ângulo de rotação
r_allegorie = triple(0.0,1.0,0.0)     # rotação
t_allegorie = triple(0.8,-1.0,-1.5)   # translação
s_allegorie = triple(0.1,0.1,0.1)     # escala
ka_allegorie = 0.8                    # coeficiente de reflexão ambiente 
kd_allegorie = 0.8                    # coeficiente de reflexão difusa 
ks_allegorie = 0.7                    # coeficiente de reflexão especular 
ns_allegorie = 256.0                  # expoente de reflexao especular
vi_allegorie, qtd_allegorie, id_allegorie = insert_model('modelos/allegorie_sculpture/Allegorie_modifiedobj.obj','modelos/allegorie_sculpture/allegorie.png') # vértices

Processando modelo modelos/allegorie_sculpture/Allegorie_modifiedobj.obj ...
Vértice inicial: 94752
Vértice final: 1864794


In [170]:

#------MATERNIDADE SCULPTURE------#

angle_maternidade = 0.0                 # ângulo de rotação
r_maternidade = triple(0.0,1.0,0.0)     # rotação
t_maternidade = triple(-8.0,-0.7,-2.6)  # translação
s_maternidade = triple(1.0,1.0,1.0)     # escala
ka_maternidade = 0.8                    # coeficiente de reflexão ambiente 
kd_maternidade = 0.8                    # coeficiente de reflexão difusa 
ks_maternidade = 0.7                    # coeficiente de reflexão especular 
ns_maternidade = 256.0                  # expoente de reflexao especular
vi_maternidade, qtd_maternidade, id_maternidade = insert_model('modelos/maternidade_sculpture/maternidade_modified.obj','modelos/maternidade_sculpture/maternidade.jpg') # vértices

Processando modelo modelos/maternidade_sculpture/maternidade_modified.obj ...
Vértice inicial: 1864794
Vértice final: 1875141


In [171]:

#------PAINTS------#

ka_paint = 0.8                    # coeficiente de reflexão ambiente 
kd_paint = 0.8                    # coeficiente de reflexão difusa 
ks_paint = 0.2                    # coeficiente de reflexão especular 
ns_paint = 32.0                   # expoente de reflexao especular

angle_paint_1 = 0.0                 # ângulo de rotação
r_paint_1 = triple(0.0,1.0,0.0)     # rotação
t_paint_1 = triple(-2.0,1.5,-3.7)   # translação
s_paint_1 = triple(0.2,0.2,0.2)     # escala
vi_paint_1, qtd_paint_1, id_paint_1 = insert_model('modelos/paints/paint-01.obj','modelos/paints/paint-01.jpg') # vértices

angle_paint_2 = 0.0                    # ângulo de rotação
r_paint_2 = triple(0.0,1.0,0.0)        # rotação
t_paint_2 = triple(-0.5,1.0,-3.7)      # translação
s_paint_2 = triple(0.15,0.15,0.15)     # escala
vi_paint_2, qtd_paint_2, id_paint_2 = insert_model('modelos/paints/paint-02.obj','modelos/paints/paint-02.jpg') # vértices

Processando modelo modelos/paints/paint-01.obj ...
Vértice inicial: 1875141
Vértice final: 1875243
Processando modelo modelos/paints/paint-02.obj ...
Vértice inicial: 1875243
Vértice final: 1875345


In [172]:

#-------------BOOTH-------------#

angle_booth = 0.0                                                                                   # ângulo de rotação
r_booth = triple(0.0, 0.0, 1.0)                                                                     # rotação
t_booth = triple(-16.0, -1.0, 12.0)                                                                 # translação
s_booth = triple(2.0,2.0,2.0)                                                                       # escala
ka_booth = 0.7                                                                                      # coeficiente de reflexão ambiente 
kd_booth = 0.7                                                                                      # coeficiente de reflexão difusa 
ks_booth = 0.7                                                                                      # coeficiente de reflexão especular 
ns_booth = 512.0                                                                                    # expoente de reflexao especular
vi_booth, qtd_booth, id_booth = insert_model('modelos/booth/booth.obj','modelos/booth/booth.jpg')   # vértices

Processando modelo modelos/booth/booth.obj ...
Vértice inicial: 1875345
Vértice final: 1876617


In [173]:

#-------------HACHIROKU-------------#

angle_car = 0.0                                                                                               # ângulo de rotação
r_car = triple(0.0, 1.0, 0.0)                                                                                 # rotação
t_car = triple(-16.0, -1.0, 16.2)                                                                             # translação
s_car = triple(1.2,1.2,1.2)                                                                                   # escala
ka_car = 0.85                                                                                                 # coeficiente de reflexão ambiente 
kd_car = 0.85                                                                                                 # coeficiente de reflexão difusa 
ks_car = 0.9                                                                                                  # coeficiente de reflexão especular 
ns_car = 512.0                                                                                                # expoente de reflexao especular
vi_car, qtd_car, id_car = insert_model('modelos/hachiroku/hachiroku.obj','modelos/hachiroku/hachiroku.jpg')   # vértices

Processando modelo modelos/hachiroku/hachiroku.obj ...
Vértice inicial: 1876617
Vértice final: 1883019


In [174]:

#-------------LIGHT SOURCE-------------#

angle_light = 0.0                     # ângulo de rotação
r_light = triple(0.0,1.0,0.0)         # rotação
t_light = triple(0.0,6.0,0.0)         # translação
s_light = triple(0.3,0.3,0.3)         # escala
ka_light = 1.0                        # coeficiente de reflexão ambiente 
kd_light = 1.0                        # coeficiente de reflexão difusa 
ks_light = 1.0                        # coeficiente de reflexão especular 
ns_light = 10.0                       # expoente de reflexao especular
vi_light, qtd_light, id_light = insert_model('modelos/light/luz.obj','modelos/light/luz.png')  

angle_sun = 0.0                     # ângulo de rotação
r_sun = triple(0.0,0.0,1.0)         # rotação
t_sun = triple(60.0,60.0,0.0)       # translação
s_sun = triple(1.0,1.0,1.0)         # escala
ka_sun = 1.0                        # coeficiente de reflexão ambiente 
kd_sun = 1.0                        # coeficiente de reflexão difusa 
ks_sun = 1.0                        # coeficiente de reflexão especular 
ns_sun = 1000.0                     # expoente de reflexao especular
vi_sun, qtd_sun, id_sun = insert_model('modelos/light/sun.obj','modelos/light/sun.jpg')  

Processando modelo modelos/light/luz.obj ...
Vértice inicial: 1883019
Vértice final: 1883055
Processando modelo modelos/light/sun.obj ...
Vértice inicial: 1883055
Vértice final: 1980591


In [175]:
# House:
# Author: PoossyJooce

# Duck:
# Author: tantangula

# Museum (modificado):
# Author: Veerle Brugman

# Tree:
# Author: Smurrie

# Hachi-roku:
# Author: metalmorag

# License: https://creativecommons.org/licenses/by-nc-nd/4.0/

### Para enviar nossos dados da CPU para a GPU, precisamos requisitar slots.

Nós agora vamos requisitar dois slots.
* Um para enviar coordenadas dos vértices.
* Outros para enviar coordenadas de texturas.

In [176]:
# Request a buffer slot from GPU
buffer = glGenBuffers(3)


###  Enviando coordenadas de vértices para a GPU

In [177]:
vertices = np.zeros(len(vertices_list), [("position", np.float32, 3)])
vertices['position'] = vertices_list


# Upload data
glBindBuffer(GL_ARRAY_BUFFER, buffer[0])
glBufferData(GL_ARRAY_BUFFER, vertices.nbytes, vertices, GL_STATIC_DRAW)
stride = vertices.strides[0]
offset = ctypes.c_void_p(0)
loc_vertices = glGetAttribLocation(program, "position")
glEnableVertexAttribArray(loc_vertices)
glVertexAttribPointer(loc_vertices, 3, GL_FLOAT, False, stride, offset)


###  Enviando coordenadas de textura para a GPU

In [178]:
textures = np.zeros(len(textures_coord_list), [("position", np.float32, 2)]) # duas coordenadas
textures['position'] = textures_coord_list

# Upload data
glBindBuffer(GL_ARRAY_BUFFER, buffer[1])
glBufferData(GL_ARRAY_BUFFER, textures.nbytes, textures, GL_STATIC_DRAW)
stride = textures.strides[0]
offset = ctypes.c_void_p(0)
loc_texture_coord = glGetAttribLocation(program, "texture_coord")
glEnableVertexAttribArray(loc_texture_coord)
glVertexAttribPointer(loc_texture_coord, 2, GL_FLOAT, False, stride, offset)


###  Enviando dados de Iluminação a GPU

#### Dados de iluminação: vetores normais

In [179]:
normals = np.zeros(len(normals_list), [("position", np.float32, 3)]) # três coordenadas
normals['position'] = normals_list

# Upload coordenadas normals de cada vertice
glBindBuffer(GL_ARRAY_BUFFER, buffer[2])
glBufferData(GL_ARRAY_BUFFER, normals.nbytes, normals, GL_STATIC_DRAW)
stride = normals.strides[0]
offset = ctypes.c_void_p(0)
loc_normals_coord = glGetAttribLocation(program, "normals")
glEnableVertexAttribArray(loc_normals_coord)
glVertexAttribPointer(loc_normals_coord, 3, GL_FLOAT, False, stride, offset)


### Desenhando nossos modelos
* Cada modelo tem um Model para posicioná-los no mundo.
* É necessário saber qual a posição inicial e total de vértices de cada modelo
* É necessário indicar qual o ID da textura do modelo


In [180]:
# Desenha o objeto com as propriedades especificadas
def draw_object(angle, r, s, t, vi, qtd, texture_id):

    global vertices

    # aplica a matriz model
    mat_model = model(angle, r, t, s)
    loc_model = glGetUniformLocation(program, "model")
    glUniformMatrix4fv(loc_model, 1, GL_TRUE, mat_model) 

    # define o id da textura do modelo
    glBindTexture(GL_TEXTURE_2D, texture_id)

    # desenha o objeto 
    glDrawArrays(GL_TRIANGLES, vi, qtd)

    return  

In [181]:

     
# Desenha o objeto com as propriedades especificadas 

# >> ka: coeficiente de reflexao ambiente do modelo
# >> kd: coeficiente de reflexao difusa do modelo
# >> ks: coeficiente de reflexao especular do modelo
# >> ns: expoente de reflexao especular
# >> light_id: identificador para fontes de luz (opcional) 

def draw_object_ilum(angle, r, s, t, ka, kd, ks, ns, vi, qtd, texture_id, light_id = 0):
    
    global vertices, internalLight

    mat_model = model(angle,r,t,s)
    loc_model = glGetUniformLocation(program, "model")
    glUniformMatrix4fv(loc_model, 1, GL_TRUE, mat_model)
    
    loc_ka = glGetUniformLocation(program, "ka") # recuperando localizacao da variavel ka na GPU
    glUniform1f(loc_ka, ka)                      ### envia ka pra gpu
    
    loc_kd = glGetUniformLocation(program, "kd") # recuperando localizacao da variavel ka na GPU
    glUniform1f(loc_kd, kd)                      ### envia kd pra gpu    

    loc_ks = glGetUniformLocation(program, "ks") # recuperando localizacao da variavel ks na GPU
    glUniform1f(loc_ks, ks) ### envia ks pra gpu        
    
    loc_ns = glGetUniformLocation(program, "ns") # recuperando localizacao da variavel ns na GPU
    glUniform1f(loc_ns, ns) ### envia ns pra gpu        

    if(light_id >= 1):
        lightPos = "lightPos" + str(light_id)
        loc_light_pos = glGetUniformLocation(program, lightPos) # recuperando localizacao da variavel lightPos na GPU
        glUniform3f(loc_light_pos, t[0], t[1], t[2])            ### posicao da fonte de luz

    # fonte de luz interna
    if(light_id == 1):
        if(internalLight): 
            loc_il = glGetUniformLocation(program, "il1") # recuperando localizacao da variavel ns na GPU
            glUniform1f(loc_il, 1.0) 
        else:
            loc_il = glGetUniformLocation(program, "il1") # recuperando localizacao da variavel ns na GPU
            glUniform1f(loc_il, 0.0) 
    
    glBindTexture(GL_TEXTURE_2D, texture_id)   # define id da textura do modelo
    glDrawArrays(GL_TRIANGLES, vi, qtd)        # desenha o modelo

    return

In [182]:
# Desenha objeto com múltiplas texturas e com as propriedades especificadas
def draw_object_mult_texture(angle,r,s,t,vi,qtd,texture_id):

    global vertices

    # aplica a matriz model
    mat_model = model(angle, r, t, s)
    loc_model = glGetUniformLocation(program, "model")
    glUniformMatrix4fv(loc_model, 1, GL_TRUE, mat_model) 

    n = len(texture_id)

    for i in range(0,n):
        glBindTexture(GL_TEXTURE_2D, texture_id[i]) # define o id da textura do modelo
        glDrawArrays(GL_TRIANGLES, vi[i], qtd[i])   # desenha o objeto 

    return  

In [183]:
# Desenha objeto com múltiplas texturas e com as propriedades especificadas

# >> ka: coeficiente de reflexao ambiente do modelo
# >> kd: coeficiente de reflexao difusa do modelo
# >> ks: coeficiente de reflexao especular do modelo
# >> ns: expoente de reflexao especular
# >> light_id: identificador para fontes de luz (opcional) 

def draw_object_mult_texture_ilum(angle, r, s, t, ka, kd, ks, ns, vi, qtd, texture_id, light_id = 0):

    global vertices

    # aplica a matriz model
    mat_model = model(angle, r, t, s)
    loc_model = glGetUniformLocation(program, "model")
    glUniformMatrix4fv(loc_model, 1, GL_TRUE, mat_model) 

    loc_ka = glGetUniformLocation(program, "ka") # recuperando localizacao da variavel ka na GPU
    glUniform1f(loc_ka, ka)                      ### envia ka pra gpu
    
    loc_kd = glGetUniformLocation(program, "kd") # recuperando localizacao da variavel ka na GPU
    glUniform1f(loc_kd, kd)                      ### envia kd pra gpu    

    loc_ks = glGetUniformLocation(program, "ks") # recuperando localizacao da variavel ks na GPU
    glUniform1f(loc_ks, ks) ### envia ks pra gpu        
    
    loc_ns = glGetUniformLocation(program, "ns") # recuperando localizacao da variavel ns na GPU
    glUniform1f(loc_ns, ns) ### envia ns pra gpu        

    if(light_id >= 1):
        lightPos = "lightPos" + str(light_id)
        loc_light_pos = glGetUniformLocation(program, lightPos) # recuperando localizacao da variavel lightPos na GPU
        glUniform3f(loc_light_pos, t[0], t[1], t[2])            ### posicao da fonte de luz

    n = len(texture_id)

    for i in range(0,n):
        glBindTexture(GL_TEXTURE_2D, texture_id[i]) # define o id da textura do modelo
        glDrawArrays(GL_TRIANGLES, vi[i], qtd[i])   # desenha o objeto 

    return  

In [184]:
# Desenha o objeto com as propriedades especificadas
def draw_eagle(angle,r,s,t,ka,kd,ks,ns,vi,qtd,texture_id,light_id = 0):

    global vertices

    angle = math.radians(angle)

    # aplica a matriz model
    mat_model = glm.mat4(1.0)                                            # instanciando uma matriz identidade
    mat_model = glm.translate(mat_model, glm.vec3(-t[0], 0.0, -t[2]))    # aplicando translacao para a origem
    mat_model = glm.rotate(mat_model, angle, glm.vec3(r[0], r[1], r[2])) # aplicando rotacao 
    mat_model = glm.translate(mat_model, glm.vec3(t[0], t[1], t[2]))     # aplicando translacao de volta ao ponto original
       
    mat_model = glm.scale(mat_model, glm.vec3(s[0], s[1], s[2]))  # aplicando escala
    mat_model = np.array(mat_model).T                             # pegando a transposta da matriz (glm trabalha com ela invertida)

    loc_model = glGetUniformLocation(program, "model")
    glUniformMatrix4fv(loc_model, 1, GL_TRUE, mat_model) 

    loc_ka = glGetUniformLocation(program, "ka") # recuperando localizacao da variavel ka na GPU
    glUniform1f(loc_ka, ka)                      ### envia ka pra gpu
    
    loc_kd = glGetUniformLocation(program, "kd") # recuperando localizacao da variavel ka na GPU
    glUniform1f(loc_kd, kd)                      ### envia kd pra gpu    

    loc_ks = glGetUniformLocation(program, "ks") # recuperando localizacao da variavel ks na GPU
    glUniform1f(loc_ks, ks) ### envia ks pra gpu        
    
    loc_ns = glGetUniformLocation(program, "ns") # recuperando localizacao da variavel ns na GPU
    glUniform1f(loc_ns, ns) ### envia ns pra gpu

    if(light_id >= 1):
        lightPos = "lightPos" + str(light_id)
        loc_light_pos = glGetUniformLocation(program, lightPos) # recuperando localizacao da variavel lightPos na GPU
        glUniform3f(loc_light_pos, t[0], t[1], t[2])            ### posicao da fonte de luz 

    # define o id da textura do modelo
    glBindTexture(GL_TEXTURE_2D, texture_id)

    # desenha o objeto 
    glDrawArrays(GL_TRIANGLES, vi, qtd)

    return  

### Eventos para modificar a posição da câmera
* Usei as teclas A, S, D e W para movimentação no espaço tridimensional
* Usei a posição do mouse para "direcionar" a câmera

In [185]:
cameraPos   = glm.vec3(0.0,  0.0,  1.0);
cameraFront = glm.vec3(0.0,  0.0, -1.0);
cameraUp    = glm.vec3(0.0,  1.0,  0.0);
cameraBoundary = ((24.5, -24.5), (24.5, -0.7), (24.5, -24.5))

internalLight = True
ia = 1.0
loc_ks = glGetUniformLocation(program, "ia") # recuperando localizacao da variavel ia na GPU
glUniform1f(loc_ks, ia)                      # envia ia pra gpu 

polygonal_mode = False

def key_event(window,key,scancode,action,mods):
    global cameraPos, cameraFront, cameraUp, polygonal_mode, cameraBoundary, ia, internalLight
    
    cameraSpeed = 0.2
    if key == 87 and (action==1 or action==2): # tecla W
        cameraPos += cameraSpeed * cameraFront
    
    if key == 83 and (action==1 or action==2): # tecla S
        cameraPos -= cameraSpeed * cameraFront
    
    if key == 65 and (action==1 or action==2): # tecla A
        cameraPos -= glm.normalize(glm.cross(cameraFront, cameraUp)) * cameraSpeed
        
    if key == 68 and (action==1 or action==2): # tecla D
        cameraPos += glm.normalize(glm.cross(cameraFront, cameraUp)) * cameraSpeed
    
    if key == 76 and (action==1 or action==2): # tecla L
        internalLight = not internalLight

    if key == 80 and (action==1 or action==2) and ia < 1.0: # tecla P
        ia += 0.1
        loc_ia = glGetUniformLocation(program, "ia") # recuperando localizacao da variavel ia na GPU
        glUniform1f(loc_ia, ia) ### envia ia pra gpu 

    if key == 85 and (action==1 or action==2) and ia > 0.0: # tecla U
        ia -= 0.1
        loc_ia = glGetUniformLocation(program, "ia") # recuperando localizacao da variavel ia na GPU
        glUniform1f(loc_ia, ia) ### envia ia pra gpu 
        
    if key == 79 and action==1 and polygonal_mode==True:
        polygonal_mode=False
    else:
        if key == 79 and action==1 and polygonal_mode==False:
            polygonal_mode=True
                
    if cameraPos[0] > cameraBoundary[0][0]:
        cameraPos[0] = cameraBoundary[0][0]
    elif cameraPos[0] < cameraBoundary[0][1]:
        cameraPos[0] = cameraBoundary[0][1]
        
    if cameraPos[1] > cameraBoundary[1][0]:
        cameraPos[1] = cameraBoundary[1][0]
    elif cameraPos[1] < cameraBoundary[1][1]:
        cameraPos[1] = cameraBoundary[1][1]
        
    if cameraPos[2] > cameraBoundary[2][0]:
        cameraPos[2] = cameraBoundary[2][0]
    elif cameraPos[2] < cameraBoundary[2][1]:
        cameraPos[2] = cameraBoundary[2][1]
        
firstMouse = True
yaw = -90.0 
pitch = 0.0
lastX =  largura/2
lastY =  altura/2

def mouse_event(window, xpos, ypos):
    global firstMouse, cameraFront, yaw, pitch, lastX, lastY
    if firstMouse:
        lastX = xpos
        lastY = ypos
        firstMouse = False

    xoffset = xpos - lastX
    yoffset = lastY - ypos
    lastX = xpos
    lastY = ypos

    sensitivity = 0.3 
    xoffset *= sensitivity
    yoffset *= sensitivity

    yaw += xoffset;
    pitch += yoffset;

    
    if pitch >= 90.0: pitch = 90.0
    if pitch <= -90.0: pitch = -90.0

    front = glm.vec3()
    front.x = math.cos(glm.radians(yaw)) * math.cos(glm.radians(pitch))
    front.y = math.sin(glm.radians(pitch))
    front.z = math.sin(glm.radians(yaw)) * math.cos(glm.radians(pitch))
    cameraFront = glm.normalize(front)

    
glfw.set_key_callback(window,key_event)
glfw.set_cursor_pos_callback(window, mouse_event)


### Matrizes Model, View e Projection

Teremos uma aula específica para entender o seu funcionamento.

In [186]:
def model(angle, r, t, s):
    
    angle = math.radians(angle)
    
    matrix_transform = glm.mat4(1.0)                                                   # instanciando uma matriz identidade
    matrix_transform = glm.rotate(matrix_transform, angle, glm.vec3(r[0], r[1], r[2])) # aplicando rotacao 
    matrix_transform = glm.translate(matrix_transform, glm.vec3(t[0], t[1], t[2]))     # aplicando translacao    
    matrix_transform = glm.scale(matrix_transform, glm.vec3(s[0], s[1], s[2]))         # aplicando escala
    matrix_transform = np.array(matrix_transform).T                                    # pegando a transposta da matriz (glm trabalha com ela invertida)
    
    return matrix_transform

def view():
    global cameraPos, cameraFront, cameraUp
    mat_view = glm.lookAt(cameraPos, cameraPos + cameraFront, cameraUp);
    mat_view = np.array(mat_view)
    return mat_view

def projection():
    global altura, largura
    # perspective parameters: fovy, aspect, near, far
    mat_projection = glm.perspective(glm.radians(45.0), largura/altura, 0.1, 1000.0)
    mat_projection = np.array(mat_projection)    
    return mat_projection
    

### Nesse momento, nós exibimos a janela!


In [187]:
glfw.show_window(window)
glfw.set_cursor_pos(window, lastX, lastY)

### Loop principal da janela.
Enquanto a janela não for fechada, esse laço será executado. É neste espaço que trabalhamos com algumas interações com a OpenGL.

In [188]:
glEnable(GL_DEPTH_TEST) ### importante para 3D

global angle_eagle, angle_sun, angle_pokeball, t_pokeball

while not glfw.window_should_close(window):

    #atualização de variáveis para animação dos modelos
    angle_eagle -= 1.0
    if(angle_eagle == -360.0): angle_eagle = 0.0

    angle_sun -= 1.0
    if(angle_sun == -360.0): angle_sun = 0.0

    angle_pokeball += 2.0
    if(angle_pokeball == 360.0): angle_pokeball = 0.0
    t_pokeball[0] = math.cos(math.radians(angle_pokeball))
    t_pokeball[2] = math.sin(math.radians(angle_pokeball))

    glfw.poll_events() 
    
    glClear(GL_COLOR_BUFFER_BIT | GL_DEPTH_BUFFER_BIT)
    
    glClearColor(0.0, 0.0, 0.0, 1.0)
    
    if polygonal_mode == True: glPolygonMode(GL_FRONT_AND_BACK,GL_LINE)
    if polygonal_mode == False: glPolygonMode(GL_FRONT_AND_BACK,GL_FILL)
    
    #sky
    draw_object_ilum(angle_sky,r_sky,s_sky,t_sky,ka_sky,kd_sky,ks_sky,ns_sky,vi_sky,qtd_sky,id_sky)
    #terrain
    draw_object_ilum(angle_grass,r_grass,s_grass,t_grass,ka_grass,kd_grass,ks_grass,ns_grass,vi_grass,qtd_grass,id_grass)
    draw_object_ilum(angle_street,r_street,s_street,t_street,ka_street,kd_street,ks_street,ns_street,vi_street,qtd_street,id_street)
    #birdbath
    draw_object_ilum(angle_birdbath,r_birdbath,s_birdbath,t_birdbath,ka_birdbath,kd_birdbath,ks_birdbath,ns_birdbath,vi_birdbath,qtd_birdbath,id_birdbath) 
    #eagle
    draw_eagle(angle_eagle,r_eagle,s_eagle,t_eagle,ka_eagle,kd_eagle,ks_eagle,ns_eagle,vi_eagle,qtd_eagle,id_eagle) 
    #museum
    draw_object_ilum(angle_museum,r_museum,s_museum,t_museum,ka_museum,kd_museum,ks_museum,ns_museum,vi_museum,qtd_museum,id_museum)
    draw_object_ilum(angle_floor_m,r_floor_m,s_floor_m,t_floor_m,ka_floor_m,kd_floor_m,ks_floor_m,ns_floor_m,vi_floor_m,qtd_floor_m,id_floor_m)
    #table
    draw_object_ilum(angle_table,r_table,s_table,t_table,ka_table,kd_table,ks_table,ns_table,vi_table,qtd_table,id_table)
    draw_object_ilum(angle_pokeball,r_pokeball,s_pokeball,t_pokeball,ka_pokeball,kd_pokeball,ks_pokeball,ns_pokeball,vi_pokeball,qtd_pokeball,id_pokeball,1)
    #duck
    draw_object_ilum(angle_duck,r_duck,s_duck,t_duck,ka_duck,kd_duck,ks_duck,ns_duck,vi_duck,qtd_duck,id_duck)
    #tree
    draw_object_mult_texture_ilum(angle_tree,r_tree,s_tree,t_tree,ka_tree,kd_tree,ks_tree,ns_tree,vi_tree,qtd_tree,id_tree)
    #seeds
    draw_object_ilum(angle_seeds,r_seeds,s_seeds,t_seeds,ka_seeds,kd_seeds,ks_seeds,ns_seeds,vi_seeds,qtd_seeds,id_seeds)
    #apples
    draw_object_ilum(angle_apple[0],r_apple[0],s_apple[0],t_apple[0],ka_apple,kd_apple,ks_apple,ns_apple,vi_apple,qtd_apple,id_apple)
    draw_object_ilum(angle_apple[1],r_apple[1],s_apple[1],t_apple[1],ka_apple,kd_apple,ks_apple,ns_apple,vi_apple,qtd_apple,id_apple)
    draw_object_ilum(angle_apple[2],r_apple[2],s_apple[2],t_apple[2],ka_apple,kd_apple,ks_apple,ns_apple,vi_apple,qtd_apple,id_apple)
    #campfire
    draw_object_ilum(angle_campfire,r_campfire,s_campfire,t_campfire,ka_campfire,kd_campfire,ks_campfire,ns_campfire,vi_campfire,qtd_campfire,id_campfire)
    #sculptures
    draw_object_ilum(angle_allegorie,r_allegorie,s_allegorie,t_allegorie,ka_allegorie,kd_allegorie,ks_allegorie,ns_allegorie,vi_allegorie,qtd_allegorie,id_allegorie)
    draw_object_ilum(angle_maternidade,r_maternidade,s_maternidade,t_maternidade,ka_maternidade,kd_maternidade,ks_maternidade,ns_maternidade,vi_maternidade,qtd_maternidade,id_maternidade)
    #paints
    draw_object_ilum(angle_paint_1,r_paint_1,s_paint_1,t_paint_1,ka_paint,kd_paint,ks_paint,ns_paint,vi_paint_1,qtd_paint_1,id_paint_1)
    draw_object_ilum(angle_paint_2,r_paint_2,s_paint_2,t_paint_2,ka_paint,kd_paint,ks_paint,ns_paint,vi_paint_2,qtd_paint_2,id_paint_2)
    #booth
    draw_object_ilum(angle_booth,r_booth,s_booth,t_booth,ka_booth,kd_booth,ks_booth,ns_booth,vi_booth,qtd_booth,id_booth)
    #hachiroku
    draw_object_ilum(angle_car,r_car,s_car,t_car,ka_car,kd_car,ks_car,ns_car,vi_car,qtd_car,id_car)

    #light sources
    #draw_object_ilum(angle_light,r_light,s_light,t_light,ka_light,kd_light,ks_light,ns_light,vi_light,qtd_light)
    draw_eagle(angle_sun,r_sun,s_sun,t_sun,ka_sun,kd_sun,ks_sun,ns_sun,vi_sun,qtd_sun,id_sun,2)

    mat_view = view()
    loc_view = glGetUniformLocation(program, "view")
    glUniformMatrix4fv(loc_view, 1, GL_FALSE, mat_view)

    mat_projection = projection()
    loc_projection = glGetUniformLocation(program, "projection")
    glUniformMatrix4fv(loc_projection, 1, GL_FALSE, mat_projection)    
    
    glfw.swap_buffers(window)

glfw.terminate()